# Задание

Реализуйте ResidualBlok, применяемый в ResNet. В решении
используйте свертки с ядром 3, padding = 1 и stride = 1. Число каналов в
тензоре не должно поменяться.


In [3]:
import torch          # Основная библиотека PyTorch для работы с тензорами и вычислениями
import torch.nn as nn # Модуль с определениями нейронных сетевых слоев и архитектур
import torch.nn.functional as F  # Модуль с функциональными API, например активациями, потерями, и пр.

In [4]:

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        # Первый сверточный слой (Conv2d):
        # - channels: число входных и выходных каналов (т.к. слои сохраняют размерность)
        # - kernel_size=3: размер ядра свертки 3x3, что хорошо для захвата локальных признаков
        # - stride=1: шаг свертки равен 1, значит размер выхода не уменьшается
        # - padding=1: добавляем 1 пиксель по краям, чтобы сохранить размер входа и выхода одинаковым
        # - bias=False: смещение (биас) не используется, т.к. есть нормализация или для упрощения модели
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1, bias=False)


        # Второй сверточный слой с теми же параметрами, чтобы глубже обработать признаки
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1, bias=False)

    def forward(self, x):
        # Функция forward описывает, как данные проходят через блок при прямом проходе
        # x — входной тензор с формой [batch_size, channels, height, width]

        # Пропускаем вход x через первый сверточный слой, затем применяем ReLU активацию,
        # которая добавляет нелинейность и "обнуляет" отрицательные значения
        out = F.relu(self.conv1(x))

        # Далее пропускаем полученный результат через второй сверточный слой без активации,
        # чтобы получить новые признаки
        out = self.conv2(out)

        # Добавляем исходный вход x к выходу второго слоя — это называется "пропускной путь" или skip connection,
        # который помогает избежать проблемы исчезающего градиента и облегчает обучение глубоких сетей
        out += x
        # Применяем ReLU активацию к сумме, чтобы сохранить нелинейность после добавления
        out = F.relu(out)

        # Возвращаем итоговый тензор с теми же размерами, что и вход
        return out

# Пример использования ResidualBlock:
res_block = ResidualBlock(64)
# Создаем случайный входной тензор:
# torch.randn — генерирует тензор с нормальным распределением
# Размерность: 1 — размер батча (кол-во изображений в пакете)
# 64 — количество каналов (например, признаков)
# 32x32 — высота и ширина изображения
input_tensor = torch.randn(1, 64, 32, 32)

# Передаем входной тензор через residual блок
output_tensor = res_block(input_tensor)

# Печатаем размерность выходного тензора:
# Ожидаем ту же форму, что и у входа: (1, 64, 32, 32)
print(output_tensor.shape)



torch.Size([1, 64, 32, 32])
